In [2]:
import requests
from bs4 import BeautifulSoup
import sys
from itertools import zip_longest
from collections import OrderedDict
import pandas as pd

## step 1. read file(rs IDs)

In [14]:
file='testrs.txt'

In [18]:
gsa=pd.read_csv(file)

In [19]:
gsa.head()

,rsID
0,rs1799971
1,rs1815739
2,rs1234
3,rs91215
4,rs1333049


## step 2. make rawHTML column with rsID, function

In [20]:
def gethtml(TITLE):
    S = requests.Session()
    
    URL = "https://bots.snpedia.com/api.php"


    PARAMS = {
    'action': "parse",
    'page': TITLE,
    'section': 0,
    'format': "json"
    }
    print(TITLE)
    try:
        res = S.get(url=URL, params=PARAMS)
        data = res.json()
        wikitext = data['parse']['text']['*']
        lines = wikitext.split('|-')
        result = ''.join(lines)
        return result
    except (KeyError):
        print("CHECK THE rsID : no data")
        return ''
    except Exception as e:
        print("unexpected error, ", e)
        return ''

### apply function

In [21]:
gsa['rawHTML']=gsa['rsID'].apply(gethtml)

rs1799971
rs1815739
rs1234
rs91215
CHECK THE rsID : no data
rs1333049
rs97846
CHECK THE rsID : no data
rs9939609


### result

In [22]:
gsa['rsID'].size

7

In [23]:
gsa = gsa.drop_duplicates(subset='rsID')

## step 3. save it as csv file

In [24]:
gsa.to_csv(file+'_DB.csv',index=False,sep='\t')

## step 4. read csv file 

In [25]:
readfile= pd.read_csv('test_DB.csv',sep='\t')

In [28]:
readfile = readfile.fillna('')

In [29]:
readfile

,rsID,rawHTML
0,rs1799971,"<div class=""mw-parser-output""><p><span></span>..."
1,rs1815739,"<div class=""mw-parser-output""><p><span></span>..."
2,rs1234,"<div class=""mw-parser-output""><p><span></span>..."
3,rs91215,
4,rs1333049,"<div class=""mw-parser-output""><p><span></span>..."
5,rs97846,
6,rs9939609,"<div class=""mw-parser-output""><p><span></span>..."


### used functions

In [30]:

def make_report(rsID, rawHTML):

    if len(rawHTML)==0:
        data_gene = 'N/A'
        data_table = OrderedDict()
        data_desc = ['N/A']

    else:
        print(rsID,'---')
        print(type(rawHTML))
        #replace some texts to find each data what i want
        rsID_getTable =rawHTML.replace('sortable smwtable','getdata1')
        rsID_getGENE =rsID_getTable.replace('<table border="0">\n<tbody><tr><td width="90">Gene</td>','<table border="0" class="getdata2">\n<tbody><tr><td width="90">Gene</td>')

        #makeSoup with replaced rawHTML
        soup = BeautifulSoup(rsID_getGENE, 'html.parser')

        #step 1, find gene name
        #-------------------------------------------
        try:
            data_gene = soup.find(class_='getdata2').a['title']
        except:
            data_gene = 'N/A'
        #step 2, find table as OrderedDict
        #-------------------------------------------
        try:
            table_html = soup.find(class_="getdata1")
            table_text = table_html.get_text()
            table_text = table_text.split('\n')
            #cut front & last space
            table_text = table_text[1:-1]
            column = table_text[:7]
            data = table_text[7:]
            grouped_data = list(zip_longest(*(iter(data),)*7))
            data_table = []
            for i in range(len(grouped_data)):
                data_table.append(OrderedDict((k,v) for k, v in zip(column,grouped_data[i])))
        except:
            data_table = OrderedDict()
        #step 3, find table as df
        #-------------------------------------------
        try:
            ptags = soup.find_all("p")[1:6]
            data_desc = []
            for ele in ptags:
                data_desc.append(ele.get_text())
        except:
            data_desc = ['N/A']
    return rsID, data_gene, data_table, data_desc
    
def make_orderedDicted(rs,ge,tb,ds):
    result = OrderedDict([('rsID',rs),('GENE',ge),('TABLE',tb),('DESC',ds)])
    return result


## step 5. make report

In [31]:
tmp=[]
for one,two in zip(readfile['rsID'],readfile['rawHTML']):

    rs,ge,tb,ds = make_report(one, two)
    report = make_orderedDicted(rs,ge,tb,ds)

    print('rsID - '+one +' done' )

    print('------------------------')
    tmp.append(report) 

rs1799971 ---
<class 'str'>
rsID - rs1799971 done
------------------------
rs1815739 ---
<class 'str'>
rsID - rs1815739 done
------------------------
rs1234 ---
<class 'str'>
rsID - rs1234 done
------------------------
rsID - rs91215 done
------------------------
rs1333049 ---
<class 'str'>
rsID - rs1333049 done
------------------------
rsID - rs97846 done
------------------------
rs9939609 ---
<class 'str'>
rsID - rs9939609 done
------------------------


## result

In [59]:
tmp

[OrderedDict([('rsID', 'rs1333049'),
              ('GENE', 'N/A'),
              ('TABLE',
               [OrderedDict([('', ''),
                             ('Geno', '(C;C)'),
                             ('Mag', '4'),
                             ('Summary',
                              '1.9x increased risk for coronary artery disease')]),
                OrderedDict([('', ''),
                             ('Geno', '(C;G)'),
                             ('Mag', '3'),
                             ('Summary', '1.5x increased risk for CAD')]),
                OrderedDict([('', ''),
                             ('Geno', '(G;G)'),
                             ('Mag', '0'),
                             ('Summary', 'normal')])]),
              ('DESC',
               ['This SNP has been recognized by the Coriell Personalized Medicine Collaborative ICOB.\nAdditional information is available here\n',
                'rs1333049 has been reported in a large study to be associated with heart 